### Ingestion del archivo "language.csv" 

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

In [0]:
languages_schema = StructType( fields=[
    StructField("languageId", IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])

In [0]:
#Agregamos v_file_date, ya que ahora en nuestro Data Lake, tenemos carpetas con la fecha
#Seccion carga incormental, para esta carga por ser pocos datos, sera una carga completa
languages_df = spark.read \
               .option("header", True) \
               .schema(languages_schema) \
               .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")
# languages_df = spark.read \
#               .option("header", True) \
#               .schema(languages_schema) \
#               .csv(f"{bronze_folder_path}/language.csv")

### Paso 2 - Seleccionar solo las columnas "requeridas"

In [0]:
from pyspark.sql.functions import col

In [0]:
languages_selected_df = languages_df.select(col("languageId"), col("languageName"))

#### Paso 3- Cambiar el nombre de las columnas según lo "requerido"

In [0]:
languages_renamed_df = languages_selected_df.withColumnsRenamed({"languageId":"language_id", "languageName":"language_name"})

### Paso 4 -Agregar la columna "ingestion_date" y "enviroment" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languages_final_df = add_ingestion_date(languages_renamed_df) \
                     .withColumn("enviroment",lit(v_environment)) \
                     .withColumn("file_date", lit(v_file_date)) 

### Paso 5 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
# languages_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages")

# Crear una tabla en base al data frame languages_final_df
# Creara la tabla languages en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver
#languages_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages")

#Modulo: Delta Lake

languages_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
%sql
-- Consulta de la DB a la tabla languages
SELECT * FROM  movie_silver.languages;

In [0]:
dbutils.notebook.exit("Success")